In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.min_rows = 50

In [ ]:
words_import = pd.read_csv('my.dict',header=0, names=['words'])

In [ ]:
# clean import, split plurals into non-plural and plural versions
words_import = words_import.dropna()

s_end = words_import[words_import.loc[:,'words'].str[-2:] == "'s"]
s_end_normal = pd.DataFrame(s_end.loc[:,'words'].str.replace("'s",""), columns=['words'])
s_end_plural = pd.DataFrame(s_end.loc[:,'words'].str.replace("'",""), columns=['words'])

words_import = words_import[words_import.loc[:,'words'].str[-2:] != "'s"]
words_import = pd.concat([words_import, s_end_normal, s_end_plural]).reset_index(drop=True)

In [ ]:
words_df = words_import.copy()
words_df.loc[:,'words_raw'] = words_df.loc[:,'words'].str.replace("'","")
words_df = words_df.drop_duplicates(subset=['words_raw'])
words_df.loc[:,'words_length'] = words_df.loc[:,'words_raw'].apply(lambda word: len(word))
words_df = words_df[words_df.loc[:,'words_length'] > 3]

words_df.loc[:,'first_char'] = words_df.loc[:,'words_raw'].str[0]
words_df.loc[:,'last_char'] = words_df.loc[:,'words_raw'].str[-1]
words_df.loc[:,'middle_chars'] = words_df.loc[:,'words_raw'].apply(lambda word: ''.join(sorted(word[1:-1])))

In [ ]:
matches = words_df.groupby(by=['first_char','last_char','middle_chars'], as_index=False)['words'].count().rename(columns={'words':'count'})

matches = matches[matches.loc[:,'count'] > 1].reset_index(drop=True)
matches.loc[:,'match_index'] = matches.index
matches = matches.merge(words_df, how='left', on=['first_char','last_char','middle_chars'])
matches.loc[:,'single'] = 1

In [ ]:
matches_flat = matches.set_index([matches.loc[:,'match_index'], matches.groupby('match_index')['single'].cumsum()])
matches_flat = matches_flat[['words']].copy()
matches_flat = matches_flat.unstack()
matches_flat.columns = ['word_'+str(matches_flat.columns[x][1]) for x in range(len(matches_flat.columns))]

matches_flat = matches_flat.fillna('')
count_only = matches[['match_index','count']].drop_duplicates()
matches_flat = matches_flat.merge(count_only, how='left', left_index=True, right_on='match_index').drop(
    ['match_index'], axis=1).reset_index(drop=True)

In [ ]:
first_letter ='c'
top_x = 50

if first_letter == '':
    summary = matches_flat.copy()
else:
    summary = matches_flat[matches_flat.loc[:,'word_1'].str.lower().str[0] == first_letter]
summary.sort_values('count', ascending=False).head(top_x)